In [5]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from typing import List, Tuple

from IPython.core.display import display, HTML

from datasets import load_metric, load_from_disk, Dataset, DatasetDict, concatenate_datasets

from pororo import Pororo
import random
import csv


In [6]:
ner = Pororo(task="ner", lang="ko")

eng_to_kor = {'PERSON' : '사람', 'LOCATION' : '장소', 'ORGANIZATION' : '조직', 'ARTIFACT' : '인공물',
            'DATE' : '날짜', 'TIME' : '시간', 'CIVILIZATION' : '문명', 'ANIMAL' : '동물',
            'PLANT' : '식물', 'QUANTITY' : '수량', 'STUDY_FIELD' : '분야', 'THEORY' : '이론',
            'EVENT' : '사건', 'MATERIAL' : '물질', 'TERM' : '용어', 
            'OCCUPATION' : '직업', 'COUNTRY' : '국가', 'CITY' : '도시', 'DISEASE' :'질병'} # 여기서부터 새로 추가한 태그

In [7]:

# 최대 2개만 !
def random_aug(ner_sent:List[Tuple[str, str]], tag_pos:List[bool], min_token_num:int = 2) -> str:  # 한 문장을 단위로
    # 1, 2, or 0.5
    # prob = min_token_num if min_token_num >= tag_num else 1 / min_token_num
    
    tag_pos_num = sum(tag_pos)
    if tag_pos_num > min_token_num: # 0, 1
        tag_pos = np.array(tag_pos)
        tag_pos_idx = np.where(tag_pos)  # get index if "True"; 2D?
        tag_pos_false = np.random.choice(tag_pos_idx[0], tag_pos_num - min_token_num, replace=False)
        tag_pos[tag_pos_false] = False

    result = ''
    # 단어마다 tag 체크 후 결합
    for item, add_tag in zip(ner_sent, tag_pos):
        # 태그 추가
        if add_tag and item[1] != 'O':
            result += f'({eng_to_kor[item[1]]}) '
        # 단어 추가
        result += item[0]      
    return result

In [16]:
def data_augmentation(examples):
    qst = examples["question"]
    result = []
    for q in qst :
        ner_question = ner(q)  # list(tuple(word, tag))    
        cnt = 0
        tag_pos = [False] * len(ner_question)
        for idx, word in enumerate(ner_question):
            if word[1] != 'O':
                tag_pos[idx] = True
                cnt += 1
        result.append(random_aug(ner_question, tag_pos, min_token_num=2))
    examples["question"] = result
    
    return examples

In [24]:
datasets = load_from_disk("/opt/ml/data/train_dataset")
train_dataset = pd.DataFrame(datasets["train"])

datasets_to_aug = load_from_disk("/opt/ml/data/train_dataset")
train_dataset_to_aug = pd.DataFrame(datasets_to_aug["train"])

# train_dataset_to_aug = train_dataset_to_aug.map(data_augmentation)

# select
train_dataset_to_aug = train_dataset_to_aug[:100]
# train_dataset_to_aug = train_dataset_to_aug.map(data_augmentation)

In [25]:
train_dataset_to_aug = data_augmentation(train_dataset_to_aug)
train_dataset_to_aug

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,(문명) 대통령을 포함한 (국가) 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293,42
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,"{'answer_start': [212], 'text': ['《경영의 실제》']}",51638,2873
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,(사람) 강희제가 (날짜) 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028,230
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",(날짜) 11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"{'answer_start': [625], 'text': ['중국']}",34146,992
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,"{'answer_start': [30], 'text': ['4개']}",47334,548
...,...,...,...,...,...,...,...
95,장치기,"편을 나누는데 마을끼리 할 경우 윗마을·아랫마을 또는 동부·서부로 나누어 하고, 같...",(수량) 한 마을 안에서 팀을 가를 경우에 사용하는 물건은?,mrc-0-005323,"{'answer_start': [58], 'text': ['장치게']}",36696,3468
96,피아노 소나타,현재 피아노 소나타로 연주되는 작품 중에는 피아노의 전신인 하프시코드 등의 악기를 ...,(인공물) 소나타의 시초가 된 연습곡을 만든 사람은?,mrc-1-001329,"{'answer_start': [61], 'text': ['도메니코 스카를라티']}",59379,867
97,김대중 내란음모 조작 사건,로널드 레이건 대통령은 캐스퍼 와인버거 미 국방부 장관과 그레그 전 대사를 한국에 ...,김대중을 구하기 위해 캐스퍼 와인버거 미 국방부 장관과 (사람) 그레그 전 (직업)...,mrc-0-001969,"{'answer_start': [0], 'text': ['로널드 레이건 대통령']}",6855,1299
98,텀블벅,텀블벅(tumblbug)의 특징은 역시 다양한 분야에 다양한 시도들을 적극 권장하며...,정해진 기간동안 목표 금액에 도달하지 못할 경우 돈을 받지 못하는 사람은?,mrc-0-003303,"{'answer_start': [380], 'text': ['창작자']}",33445,2172


In [26]:
final_dataset = pd.concat([train_dataset, train_dataset_to_aug])

In [28]:
len(final_dataset)

4052

In [29]:
final_dataset.save_to_disk('./data/test_dasol')

AttributeError: 'DataFrame' object has no attribute 'save_to_disk'